In [1]:
import pandas as pd
import plotly.express as plx
import plotly.graph_objects as go

In [2]:
df = pd.read_csv("/content/sales_data.csv")
df.head()

,date,warehouse,client_type,product_line,quantity,unit_price,total,payment
0,2021-06-01,Central,Retail,Miscellaneous,8,16.85,134.83,Credit card
1,2021-06-01,North,Retail,Breaking system,9,19.29,173.61,Cash
2,2021-06-01,North,Retail,Suspension & traction,8,32.93,263.45,Credit card
3,2021-06-01,North,Wholesale,Frame & body,16,37.84,605.44,Transfer
4,2021-06-01,Central,Retail,Engine,2,60.48,120.96,Credit card


In [3]:
df['date'] = pd.to_datetime(df['date'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          1000 non-null   datetime64[ns]
 1   warehouse     1000 non-null   object        
 2   client_type   1000 non-null   object        
 3   product_line  1000 non-null   object        
 4   quantity      1000 non-null   int64         
 5   unit_price    1000 non-null   float64       
 6   total         1000 non-null   float64       
 7   payment       1000 non-null   object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 62.6+ KB


In [4]:
df.payment.describe()

count            1000
unique              3
top       Credit card
freq              659
Name: payment, dtype: object

In [5]:
df.describe()

,quantity,unit_price,total
count,1000.000000,1000.000000,1000.000000
mean,9.395000,30.322040,289.113000
std,9.659207,12.256488,345.227596
min,1.000000,10.030000,10.350000
25%,4.000000,21.085000,93.687500
50%,6.500000,28.570000,178.360000
75%,10.000000,37.917500,321.690000
max,40.000000,66.620000,2546.330000


In [6]:
payment_sales = df.groupby('payment')['total'].sum().sort_values(ascending=False).to_frame()
payment_sales

,total
payment,
Transfer,159642.33
Credit card,110271.57
Cash,19199.10


In [7]:
prod_line_avg_price = df.groupby('product_line')['unit_price'].mean().sort_values(ascending=False).to_frame()
prod_line_avg_price

,unit_price
product_line,
Engine,60.091803
Frame & body,42.832229
Suspension & traction,33.969868
Electrical system,25.585130
Miscellaneous,22.810738
Breaking system,17.740522


In [8]:
payment_sales = df.groupby('payment')['total'].sum().sort_values(ascending=False)
trace = plx.bar(
    x=payment_sales.index,
    y=payment_sales.values,
    # color=payment_sales.values,
    title='Total sales for each payment method',
    labels={'x':'Payment Method', 'y':'Total Sales ($)'},
    text_auto='.2s', width= 500
)
trace.update_yaxes(showticklabels=False)
trace.update_traces(textposition='outside', cliponaxis=False)
trace.show()
prod_line_avg_price = df.groupby('product_line')['unit_price'].mean().sort_values(ascending=False)
trace = plx.bar(
    x=prod_line_avg_price.index,
    y=prod_line_avg_price.values,
    title='Average unit price for each product line',
    text_auto=True, width= 900,
    labels={'x':'Product Line', 'y':'Average unit price ($)'}
)
trace.update_yaxes(showticklabels=False)
trace.update_traces(textposition='outside', cliponaxis=False)
trace.show()

In [9]:
client_typ_avg_qty = df.groupby('client_type')['total'].mean().sort_values(ascending=False)
trace = plx.bar(
    x=client_typ_avg_qty.index,
    y=client_typ_avg_qty.values,
    text_auto='.2s',
    labels={'x':'Client Type', 'y':'Average Purchase($)'},
    title='Average purchase value by client type'
)
trace.update_yaxes(showticklabels=False)
trace.update_traces( textposition='outside', cliponaxis=False)
trace.update_layout(width=500)

In [10]:
product_line_purchase = df.groupby('product_line')['total'].sum().sort_values(ascending=False)
trace = plx.line(
        x=product_line_purchase.index,
        y=product_line_purchase.values,
        text=product_line_purchase.values,
        markers=True,
        title='Total purchase value by product line',
        labels={'x':'', 'y':'Total purchase ($)'},
        width=800
)
trace.update_traces(texttemplate ="%{text:.2s}", textposition='bottom left', cliponaxis=False)
trace.update_yaxes(showticklabels=False)
trace

In [11]:
days = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']
wkday = df.set_index('date')
wkday = wkday.groupby(by=wkday.index.day_name())['quantity'].sum().reindex(days)
trace = plx.line(wkday,
                x=wkday.index,
                y=wkday.values, markers=True,
                text=wkday.values, width=700,
                title='Total quantity bought by day',
                labels={'y':'sum of quantity', 'date':''}
                )
trace.update_yaxes(showticklabels=False)
trace.update_traces(texttemplate="%{text:2s}", textposition='top center', cliponaxis=False)
trace

Findings

1.Transfer is the most used payment method with 55.2% of total sales.

2.The product Engine, has the highest average unit price, approximately $60.

3.The average purchase value for wholesale client is high compared to retail client.

4.Engine, despite have a high average unit price is one of the product makind less sales. Suspension & traction is the product with the highest sale ($73k).

5.Fewer sales are made on Modndays while more sales are made on Tuesdays



